In [19]:
import os
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
col_Names =["userID", "gender", "age", "occupation", "starSign", "date", "text"]



In [20]:
df_train = pd.read_csv('train_raw.csv', names=col_Names)
print("read train_raw.csv succesfully")
df_test = pd.read_csv('test_raw.csv',names=col_Names)
print("read test_raw.csv succesfully")

read train_raw.csv succesfully
read test_raw.csv succesfully


In [21]:
#remove empty space and start spaces from training model
indices_to_remove=[]
index=0
for row in df_train.itertuples():
    text = str(row.text.strip())
    df_train.at[row[0],'text']=text
    if len(text)==0:
        indices_to_remove.append(index)
    index+=1
df_train=df_train.drop(df_train.index[indices_to_remove])
df_train=df_train.reset_index()

In [22]:
#groups by the same user ID instance for testing and training and concatenate the string of text of same ids.
df_train = (df_train.groupby(['userID','age'])['text'].apply(' '.join).reset_index())
df_test = (df_test.groupby(['userID','age'])['text'].apply(' '.join).reset_index())

In [23]:
#it converts class of ranges to predict to unique number
df_train.loc[df_train.age.isin([14,15,16]),'age'] = 1
df_train.loc[df_train.age.isin([24,25,26]),'age'] = 2
df_train.loc[df_train.age.isin([34,35,36]),'age'] = 3
df_train.loc[df_train.age.isin([44,45,46]),'age'] = 4


In [24]:
#sets the multinomial naive baeyes with count vectoriser and tfidf matrics
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
y=df_train['age']
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB(alpha=0.000001)),])
text_clf = text_clf.fit(df_train['text'], y)
predicted = text_clf.predict(df_test['text'])

In [25]:
#this is for reference without removal or joing of texts.
df_test_check = pd.read_csv('test_raw.csv',names=col_Names)
print("read dev_raw.csv succesfully")
df_test_check.loc[df_test_check.age.isin([14,15,16]),'age'] = 1
df_test_check.loc[df_test_check.age.isin([24,25,26]),'age'] = 2
df_test_check.loc[df_test_check.age.isin([34,35,36]),'age'] = 3
df_test_check.loc[df_test_check.age.isin([44,45,46]),'age'] = 4

read dev_raw.csv succesfully


In [26]:
#creates the dictionary of each unique user age predicted.
userID_list=list(df_test['userID'])
from collections import defaultdict
dict_predict=defaultdict(int)
for i in range(0,len(predicted)):
    dict_predict[userID_list[i]]=predicted[i]



In [27]:
#helps to create the list of all users age predicted with userIDs with same number of times as orignal.
new_pred_nb=[]
for i in range(0,len(df_test_check)):  
    if dict_predict[df_test_check.loc[i]['userID']] ==1:
        new_pred_nb.append((df_test_check.loc[i]['userID'],"14-16"))
    if dict_predict[df_test_check.loc[i]['userID']] ==2:
        new_pred_nb.append((df_test_check.loc[i]['userID'],"24-26"))
    if dict_predict[df_test_check.loc[i]['userID']]==3:
        new_pred_nb.append((df_test_check.loc[i]['userID'],"34-36"))
    if dict_predict[df_test_check.loc[i]['userID']] ==4:
        new_pred_nb.append((df_test_check.loc[i]['userID'],"44-46"))

In [28]:
from collections import defaultdict

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
C=[13]

for i in C:
    y=df_train['age']
    text_clf_lg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                                  ('clf-svm',LogisticRegression(C=i)),])

    _ = text_clf_lg.fit(df_train['text'], y)
    predicted_lg = text_clf_lg.predict(df_test['text'])
    userID_list=list(df_test['userID'])
    dict_predict=defaultdict(int)
    for i in range(0,len(df_test['userID'])):
        dict_predict[userID_list[i]]=predicted_lg[i]   
new_pred_lg=[]
for i in range(0,len(df_test_check)):
    if dict_predict[df_test_check.loc[i]['userID']] ==1:
        new_pred_lg.append((df_test_check.loc[i]['userID'],"14-16"))
    if dict_predict[df_test_check.loc[i]['userID']] ==2:
        new_pred_lg.append((df_test_check.loc[i]['userID'],"24-26"))
    if dict_predict[df_test_check.loc[i]['userID']]==3:
        new_pred_lg.append((df_test_check.loc[i]['userID'],"34-36"))
    if dict_predict[df_test_check.loc[i]['userID']] ==4:
        new_pred_lg.append((df_test_check.loc[i]['userID'],"44-46"))    

In [30]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                      alpha=1e-4, max_iter=5, random_state=42)),])
_ = text_clf_svm.fit(df_train['text'], y)
predicted_svm = text_clf_svm.predict(df_test['text'])


In [31]:
userID_list=list(df_test['userID'])
    

dict_predict=defaultdict(int)
for i in range(0,len(predicted)):
    dict_predict[userID_list[i]]=predicted_svm[i]

new_pred_svm=[]
for i in range(0,len(df_test_check)):
   
     
    if dict_predict[df_test_check.loc[i]['userID']] ==1:
        new_pred_svm.append((df_test_check.loc[i]['userID'],"14-16"))
    if dict_predict[df_test_check.loc[i]['userID']] ==2:
        new_pred_svm.append((df_test_check.loc[i]['userID'],"24-26"))
    if dict_predict[df_test_check.loc[i]['userID']]==3:
        new_pred_svm.append((df_test_check.loc[i]['userID'],"34-36"))
    if dict_predict[df_test_check.loc[i]['userID']] ==4:
        new_pred_svm.append((df_test_check.loc[i]['userID'],"44-46"))



In [10]:
# #decision Tree
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from matplotlib import pyplot as plt
# import seaborn as sns
# # %matplotlib inlin
# from sklearn.metrics import accuracy_score
# #best split at 1700

# text_clf_tree = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('c',DecisionTreeClassifier(min_samples_split =1700)),])

# dt = text_clf_tree.fit(df_train['text'], y)
# predicted_tree = text_clf_tree.predict(df_test['text'])
# np.mean(predicted_tree == df_test['age'])
# userID_list=list(df_test['userID'])


# dict_predict=defaultdict(int)
# for i in range(0,len(predicted)):
#     dict_predict[userID_list[i]]=predicted_tree[i]




In [ ]:


# new_pred_tree=[]
# for i in range(0,len(df_test_check)):
   
     
#     if dict_predict[df_test_check.loc[i]['userID']] ==1:
#         new_pred_tree.append((df_test_check.loc[i]['userID'],"14-16"))
#     if dict_predict[df_test_check.loc[i]['userID']] ==2:
#         new_pred_tree.append((df_test_check.loc[i]['userID'],"24-26"))
#     if dict_predict[df_test_check.loc[i]['userID']]==3:
#         new_pred_tree.append((df_test_check.loc[i]['userID'],"34-36"))
#     if dict_predict[df_test_check.loc[i]['userID']] ==4:
#         new_pred_tree.append((df_test_check.loc[i]['userID'],"44-46"))



In [32]:
ids=[x[0] for x in new_pred_lg]
li_lg=[x[1] for x in new_pred_lg]
li_svm=[x[1] for x in new_pred_svm]
# li_tree=[x[1] for x in new_pred_tree] can add later but not good for such training model.
li_nb=[x[1] for x in new_pred_nb]


In [33]:
result_age=[str() for c in 'c' * len(li_lg)]
import operator
#currenlty checks the logisitic regression but can do ensamble if we have better strategy by just adding list and stategy
#right now we have added +1 voting to predict 
df_result = pd.DataFrame(np.column_stack([li_lg,result_age]),columns=["lg","result"])

# df_result = pd.DataFrame(np.column_stack([new_predicted_lg,new_predicted_lr,new_predicted,new_predicted_svm,result_age]),columns=["lg","lr","nb","svm","result"])





for row in df_result.itertuples():
    counter_dict = {"14-16":0, "24-26":0, "34-36":0,"44-46":0}
    for model in df_result.columns.values[:-1]:
        


        model_prediction = df_result.at[row[0], model]

        counter_dict[model_prediction] += 1
   
    df_result.at[row[0],'result']=max(counter_dict.items(), key=operator.itemgetter(1))[0]




df_train_check = pd.read_csv('test_submit.csv')
ids=df_train_check['Id']
df_en = pd.DataFrame(np.column_stack([ids,df_result['result']]),columns=["Id","Prediction"])
df_en.to_csv("ML_P2.csv",index=False)


1

True

1391

In [25]:
# def normalize_document(doc):
#     # lower case and remove special characters\whitespaces

#     doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
#     doc = doc.lower()
#     doc = doc.strip()
#     # tokenize document
#     tokens = wpt.tokenize(doc)
#     # filter stopwords out of document

#     filtered_tokens = [token for token in tokens if token not in stop_words]
    
    
#     # re-create document from filtered tokens
#     doc = ' '.join(filtered_tokens)

#     return doc


# df_train = pd.read_csv('train_raw.csv', names=col_Names)
# print("read train_raw.csv succesfully")
# df_test = pd.read_csv('dev_raw.csv',names=col_Names)
# print("read dev_raw.csv succesfully")

In [26]:
# #normalise the training datFrame
# for row in df_train.itertuples():
#     df_train.at[row[0],'text']=normalize_document(df_train.at[row[0],'text'])

In [27]:
# #normalise the testing datFrame
# for row in df_test.itertuples():
#     df_test.at[row[0],'text']=normalize_document(df_test.at[row[0],'text'])
    

In [28]:
# #remove empty space and start spaces from normalise train dataFrame
# indices_to_remove=[]
# index=0
# for row in df_train.itertuples():
#     text = str(row.text.strip())
#     df_train.at[row[0],'text']=text
#     if len(text)==0:
#         indices_to_remove.append(index)
#     index+=1
# df_train=df_train.drop(df_train.index[indices_to_remove])
# df_train=df_train.reset_index()

In [29]:
# df_train.loc[df_train.age.isin([14,15,16]),'age'] = 1
# df_train.loc[df_train.age.isin([24,25,26]),'age'] = 2
# df_train.loc[df_train.age.isin([34,35,36]),'age'] = 3
# df_train.loc[df_train.age.isin([44,45,46]),'age'] = 4
# # df_test = df_test[((df_test.age>= 14) & (df_test.age<= 16)) |((df_test.age>= 24) & (df_test.age<= 26))
# #                   |((df_test.age>= 34) & (df_test.age<= 36)) |((df_test.age>= 44) & (df_test.age<= 46))]
# df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
# df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
# df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
# df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4
# # df_test=df_test.reset_index()

In [30]:
# from sklearn.metrics.pairwise import linear_kernel

# def find_similar(tfidf_matrix, index, top_n = 10):
#     cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
#     related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
#     return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]



In [31]:

# from sklearn.feature_extraction.text import CountVectorizer
# corpus_train = df_train['text'].values
# count_vectorizer = CountVectorizer()
# X_train_counts = count_vectorizer.fit_transform(corpus_train)
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_train_tfidf.shape

# corpus_test = df_test['text'].values
# count_vectorizer_test = CountVectorizer()
# X_test_counts = count_vectorizer_test.fit_transform(corpus_test)
# tfidf_transformer_test = TfidfTransformer()
# X_test_tfidf= tfidf_transformer_test.fit_transform(X_test_counts)
# X_test_tfidf.shape

In [32]:
X_test_tfidf

NameError: name 'X_test_tfidf' is not defined

In [ ]:
X_train_tfidf


In [ ]:
# print(df_train.loc[137448]['age'])
# for index,score in find_similar(X_train_tfidf,137448):
#     print(index,score,df_train.loc[index]['text'])
#     print(df_train.loc[index]['age'])
#     print("...")

In [ ]:
# print(df_train.loc[51053]['text'])
# print("....")
# print(df_train.loc[1]['text'])



In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.naive_bayes import MultinomialNB
# y=df_train['age']
# clf = MultinomialNB().fit(X_train_tfidf, y)
# text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
# text_clf = text_clf.fit(df_train['text'], y)
# predicted = text_clf.predict(df_test['text'])
# np.mean(predicted==df_test['age'])

In [ ]:
# np.mean(predicted==df_test['age'])

In [ ]:

# new_pred=[]
# for i in range(0,len(predicted)):
#     if predicted[i] ==1:
#         new_pred.append("14-16")
#     if predicted[i] ==2:
#         new_pred.append("24-26")
#     if predicted[i] ==3:
#         new_pred.append("34-36")
#     if predicted[i] ==4:
#         new_pred.append("44-46")
# new_predicted_nb=new_pred


In [ ]:
# from sklearn.linear_model import SGDClassifier
# text_clf_svm =SGDClassifier(loss='hinge', penalty='l2',
#                       alpha=1e-3, max_iter=5, random_state=42).fit(X_train_tfidf, df_train['age'])
# X_new_counts = count_vectorizer.transform(df_test['text'])
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)
# predicted_svm = text_clf_svm.predict(X_new_tfidf)

In [ ]:
# new_pred=[]
# for i in range(0,len(predicted_svm)):
#     if predicted_svm[i] ==1:
#         new_pred.append("14-16")
#     if predicted_svm[i] ==2:
#         new_pred.append("24-26")
#     if predicted_svm[i] ==3:
#         new_pred.append("34-36")
#     if predicted_svm[i] ==4:
#         new_pred.append("44-46")
# new_predicted_svm=new_pred

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report
# C=[0.1,0.5,1,5,10]
# for i in C:
#     y=df_train['age']
#     text_clf_lg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                                   ('clf-svm',LogisticRegression(C=i,tol=0.01)),])

#     _ = text_clf_lg.fit(df_train['text'], y)
#     predicted_lg = text_clf_lg.predict(df_test['text'])


#     print(np.mean(predicted_lg==df_test['age']))


# new_pred=[]
# for i in range(0,len(predicted_lg)):
#     if predicted_lg[i] ==1:
#         new_pred.append("14-16")
#     if predicted_lg[i] ==2:
#         new_pred.append("24-26")
#     if predicted_lg[i] ==3:
#         new_pred.append("34-36")
#     if predicted_lg[i] ==4:
#         new_pred.append("44-46")
    
# new_predicted_lg=new_pred

        

In [ ]:
# print(df_train.loc[217120]['age'],df_train.loc[217120]['text'])
# for index,score in find_similar(X_train_tfidf,217120):
#     print("age=",df_train.loc[index]['age'])
#     print("index=",index,"score=",score,df_train.loc[index]['text'])
    
#     print("...")

In [ ]:
1==1


In [ ]:
1==1

In [ ]:
np.mean(predicted_lg==df_test['age'])

In [ ]:

# from sklearn.svm import LinearSVC
# from sklearn.datasets import make_classification
# clf_lr =  LinearSVC(random_state=0).fit(X_train_tfidf, df_train['age'])
# X_new_counts = count_vectorizer.transform(df_test['text'])
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)
# predicted_lr = clf_lr.predict(X_new_tfidf)



# new_pred=[]
# for i in range(0,len(predicted_lr)):
#     if predicted_lr[i] ==1:
#         new_pred.append("14-16")
#     if predicted_lr[i] ==2:
#         new_pred.append("24-26")
#     if predicted_lr[i] ==3:
#         new_pred.append("34-36")
#     if predicted_lr[i] ==4:
#         new_pred.append("44-46")
# new_predicted_lr=new_pred


In [ ]:
# np.mean(predicted_lr==df_test['age'])

In [ ]:
# print(len(new_predicted))
# print(len(new_predicted_lr))

In [ ]:

# #decision Tree
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from matplotlib import pyplot as plt
# import seaborn as sns
# from sklearn.metrics import accuracy_score

# text_clf_tree = tree.DecisionTreeClassifier(min_samples_split =6500).fit(X_train_tfidf, df_train['age'])
# X_new_counts = count_vectorizer.transform(df_test['text'])
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)
# predicted_tree = text_clf_tree.predict(X_new_tfidf)
# new_pred=[]
# for i in range(0,len(predicted_tree)):
#     if predicted_tree[i] ==1:
#         new_pred.append("14-16")
#     if predicted_tree[i] ==2:
#         new_pred.append("24-26")
#     if predicted_tree[i] ==3:
#         new_pred.append("34-36")
#     if predicted_tree[i] ==4:
#         new_pred.append("44-46")
# new_predicted_tree=new_pred

In [ ]:
# df_train = pd.read_csv('test_submit.csv')
# ids=df_train['Id']


In [ ]:
# df_nb = pd.DataFrame(np.column_stack([ids,new_predicted_nb]),columns=["Id","Prediction"])
# df_lr = pd.DataFrame(np.column_stack([ids,new_predicted_lr]),columns=["Id","Prediction"])
# df_svm = pd.DataFrame(np.column_stack([ids,new_predicted_svm]),columns=["Id","Prediction"])
# df_lg = pd.DataFrame(np.column_stack([ids,new_predicted_lg]),columns=["Id","Prediction"])
# df_dt = pd.DataFrame(np.column_stack([ids,new_predicted_tree]),columns=["Id","Prediction"])



In [ ]:
# df_nb.to_csv("ss_nb.csv",index=False)
# df_lr.to_csv("ss_lr.csv",index=False)
# df_svm.to_csv("ss_svm.csv",index=False)
# df_lg.to_csv("ss_lg.csv",index=False)
# df_dt.to_csv("ss_tree.csv",index=False)


In [ ]:
# result_age=[str() for c in 'c' * len(new_predicted_lg)]


In [ ]:
# import operator
# df_result = pd.DataFrame(np.column_stack([new_predicted_lg,new_predicted_lr,new_predicted_nb,new_predicted_svm,result_age]),columns=["lg","lr","nb","svm","result"])

# # df_result = pd.DataFrame(np.column_stack([new_predicted_lg,new_predicted_lr,new_predicted,new_predicted_svm,result_age]),columns=["lg","lr","nb","svm","result"])





# for row in df_result.itertuples():
#     counter_dict = {"14-16":0, "24-26":0, "34-36":0,"44-46":0}
#     for model in df_result.columns.values[:-1]:
        


#         model_prediction = df_result.at[row[0], model]

#         counter_dict[model_prediction] += 1
   
#     df_result.at[row[0],'result']=max(counter_dict.items(), key=operator.itemgetter(1))[0]



In [ ]:
# df_en_wi_tree = pd.DataFrame(np.column_stack([ids,df_result['result']]),columns=["Id","Prediction"])
# df_dt.to_csv("ss_en_wi_t.csv",index=False)


In [ ]:





# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# clf_lg = LogisticRegression().fit(X_train_tfidf, df_train['age'])
# X_new_counts = count_vectorizer.transform(df_test['text'])
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)
# predicted_lg = clf_lg.predict(X_new_tfidf)

In [ ]:
# from sklearn.metrics import classification_report
# li= text_clf_svm.decision_function(df_test_not['text'])
# li2=text_clf_svm.predict(df_test_not['text'])
# count=0
# total=0
# for i in range(0,len(li)):
#     total += 1
    
    
#     a=li[i][0]
#     b=li[i][1]
#     c=li[i][2]
#     d=li[i][3]
#     if a < 0 and b < 0 and c < 0 and d < 0:
#         count+=1
        
# print(count)
# print(total)
# print(count/total)
    

In [ ]:
# np.mean(predicted_svm_not==y_test_not)
# df_test

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# clf_lg = LogisticRegression()
# y=df_train['age']
# text_clf_lg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('clf-svm',LogisticRegression()),])

# text_clf_lg = text_clf_lg.fit(df_train['text'], y)
# predicted_lg = text_clf_lg.predict(df_test['text'])




# # new_pred=[]
# # for i in range(0,len(predicted_lg)):
# #     if predicted_lg[i] ==1:
# #         new_pred.append("14-16")
# #     if predicted_lg[i] ==2:
# #         new_pred.append("24-26")
# #     if predicted_lg[i] ==3:
# #         new_pred.append("34-36")
# #     if predicted_lg[i] ==4:
# #         new_pred.append("44-46")
    
# # new_predicted_lg=new_pred

        

In [ ]:
# from sklearn.metrics import classification_report
# li= text_clf_lg.decision_function(df_test['text'])
# li2=text_clf_lg.predict(df_test['text'])


In [ ]:
# count=0
# total=0
# for i in range(0,len(li)):
#     total += 1
    
    
#     a=li[i][0]
#     b=li[i][1]
#     c=li[i][2]
#     d=li[i][3]
#     if a < 0 and b < 0 and c < 0 and d < 0:
#         count+=1
        
# print(count)
# print(total)
# print(count/total)

In [ ]:
# li[33502:33510]

In [ ]:
# df_test=df_test.reset_index()
# df_test

In [ ]:
# li2=[]
# li_C1=[]
# li_C2=[]
# li_C3=[]
# li_C4=[]
# for d_li in li:
#     li_C1.append(d_li[0])
#     li_C2.append(d_li[1])
#     li_C3.append(d_li[2])
#     li_C4.append(d_li[3])
# all_li=[li_C1,li_C2,li_C3,li_C4]

# for each in all_li:
  
#     i=-1
#     for elem in each:
#         i+=1
      
#         if elem<0:
#             each[i]=0
# import numpy
# for each_class in all_li:
#     print("min",numpy.min(each_class))
#     print("max",numpy.max(each_class))
#     print("mean",numpy.mean(each_class))
#     print("median",numpy.median(each_class))
#     print("class")
        
    
        
    
    


In [ ]:
1==1

In [ ]:

# from sklearn.svm import LinearSVC
# from sklearn.datasets import make_classification
# clf_lr = LinearSVC(random_state=0)
# y=df_train['age']
# text_clf_lr = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('clf-svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
#      intercept_scaling=1,  penalty='l2', random_state=0, tol=0.0001,
#      verbose=0)),])

# _ = text_clf_lr.fit(df_train['text'], y)
# predicted_lr = text_clf_lr.predict(df_test['text'])


# new_pred=[]
# for i in range(0,len(predicted_lr)):
#     if predicted_lr[i] ==1:
#         new_pred.append("14-16")
#     if predicted_lr[i] ==2:
#         new_pred.append("24-26")
#     if predicted_lr[i] ==3:
#         new_pred.append("34-36")
#     if predicted_lr[i] ==4:
#         new_pred.append("44-46")
# new_predicted_lr=new_pred


In [ ]:
# np.mean(predicted_lr==y_test)

In [ ]:
# df_test.loc[~df_test.age.isin([1,2,3,4]),'age'] = 0


In [ ]:

# #decision Tree
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from matplotlib import pyplot as plt
# import seaborn as sns
# # %matplotlib inlin
# from sklearn.metrics import accuracy_score
# li=[]
# #best split at 6500
# c = tree.DecisionTreeClassifier(min_samples_split =6500)
# text_clf_tree = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('c',DecisionTreeClassifier(min_samples_split =6500)),])

# dt = text_clf_tree.fit(df_train['text'], y)
# predicted_tree = text_clf_tree.predict(df_test['text'])
# new_pred=[]
# for i in range(0,len(predicted_tree)):
#     if predicted_tree[i] ==1:
#         new_pred.append("14-16")
#     if predicted_tree[i] ==2:
#         new_pred.append("24-26")
#     if predicted_tree[i] ==3:
#         new_pred.append("34-36")
#     if predicted_tree[i] ==4:
#         new_pred.append("44-46")
# new_predicted_tree=new_pred





In [ ]:
# np.mean(predicted_tree==y_test)

In [ ]:

# df_test.loc[df_test.age==1,'age'] = "14-16"
# df_test.loc[df_test.age==2,'age'] = "24-26"
# df_test.loc[df_test.age==3,'age'] = "34-36"
# df_test.loc[df_test.age==4,'age'] = "44-46"
# result_age=df_test['age']


In [ ]:
# result_age=[str() for c in 'c' * len(new_predicted_lg)]
# df_result = pd.DataFrame(np.column_stack([new_predicted_lg,new_predicted_lr,new_predicted_nb,new_predicted_svm,new_predicted_tree,result_age]),columns=["lg","lr","nb","svm","tree","result"])
# df_result



In [ ]:
# df_test.loc[df_test.age==1,'age'] = "14-16"
# df_test.loc[df_test.age==2,'age'] = "24-26"
# df_test.loc[df_test.age==3,'age'] = "34-36"
# df_test.loc[df_test.age==4,'age'] = "44-46"

# df_y = df_test['age']
# df_y=np.array(df_y)
# df_y

In [ ]:


# print(np.mean(df_result['lg']==df_y))
# print(np.mean(df_result['lr']==df_y))
# print(np.mean(df_result['svm']==df_y))
# print(np.mean(df_result['nb']==df_y))
# print(np.mean(df_result['tree']==df_y))


In [ ]:
# import operator
# df_result = pd.DataFrame(np.column_stack([new_predicted_lg,new_predicted_lr,new_predicted_nb,new_predicted_svm,new_predicted_tree,result_age]),columns=["lg","lr","nb","svm","tree","result"])





# for row in df_result.itertuples():
#     counter_dict = {"14-16":0, "24-26":0, "34-36":0,"44-46":0}
#     for model in df_result.columns.values[:-1]:
        


#         model_prediction = df_result.at[row[0], model]

#         counter_dict[model_prediction] += 1
   
#     df_result.at[row[0],'result']=max(counter_dict.items(), key=operator.itemgetter(1))[0]



In [ ]:
# # y_test = df_result['result']
# df_y = df_test['age']
# df_y=np.array(df_y)

# np.mean(df_y==df_result['result'])

In [ ]:

# from sklearn.svm import LinearSVC
# from sklearn.datasets import make_classification
# clf_lr = LinearSVC(random_state=0)
# y=df_train['age']
# text_clf_lr = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('clf-svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
#      intercept_scaling=1,  penalty='l2', random_state=0, tol=0.0001,
#      verbose=0)),])

# _ = text_clf_lr.fit(df_train['text'], y)
# predicted_lr = text_clf_lr.predict(df_test['text'])


# df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
# df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
# df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
# df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4
# y_test=df_test['age']
# for i in range(0,len(predicted_lr)-1):
#     if predicted_lr[i] in [14,15,16]:
#         predicted_lr[i]=1
#     if predicted_lr[i] in [24,25,26]:
#         predicted_lr[i]=2
#     if predicted_lr[i] in [34,35,36]:
#         predicted_lr[i]=3
#     if predicted_lr[i] in [44,45,46]:
#         predicted_lr[i]=4
        
    
# predicted_lr
# np.mean(predicted_lr == y_test)
 

In [ ]:


# df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
# df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
# df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
# df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4
# y_test=df_test['age']
# for i in range(0,len(predicted_lr)-1):
#     if predicted_lr[i] in [14,15,16]:
#         predicted_lr[i]=1
#     if predicted_lr[i] in [24,25,26]:
#         predicted_lr[i]=2
#     if predicted_lr[i] in [34,35,36]:
#         predicted_lr[i]=3
#     if predicted_lr[i] in [44,45,46]:
#         predicted_lr[i]=4
        
    
# predicted_lr
# np.mean(predicted_lr == y_test)
 

In [ ]:
# from sklearn.linear_model import LogisticRegression

# clf_lg = LogisticRegression()
# y=df_train['age']
# text_clf_lg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('clf-svm',LogisticRegression()),])

# _ = text_clf_lg.fit(df_train['text'], y)
# predicted_lg = text_clf_lg.predict(df_test['text'])


# df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
# df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
# df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
# df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4
# y_test=df_test['age']
# for i in range(0,len(predicted_lg)-1):
#     if predicted_lg[i] in [14,15,16]:
#         predicted_lg[i]=1
#     if predicted_lg[i] in [24,25,26]:
#         predicted_lg[i]=2
#     if predicted_lg[i] in [34,35,36]:
#         predicted_lg[i]=3
#     if predicted_lg[i] in [44,45,46]:
#         predicted_lg[i]=4
        

In [ ]:
# df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
# df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
# df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
# df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4
# y_test=df_test['age']
# for i in range(0,len(predicted_lg)-1):
#     if predicted_lg[i] in [14,15,16]:
#         predicted_lg[i]=1
#     if predicted_lg[i] in [24,25,26]:
#         predicted_lg[i]=2
#     if predicted_lg[i] in [34,35,36]:
#         predicted_lg[i]=3
#     if predicted_lg[i] in [44,45,46]:
#         predicted_lg[i]=4
        
    
# predicted_lg
# np.mean(predicted_lg == y_test)
 

In [ ]:

# #decision Tree
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from matplotlib import pyplot as plt
# import seaborn as sns
# # %matplotlib inlin
# from sklearn.metrics import accuracy_score
# li=[]
# #best split at 6500
# c = tree.DecisionTreeClassifier(min_samples_split =6500)
# text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('c',DecisionTreeClassifier(min_samples_split =6500)),])

# dt = text_clf_svm.fit(df_train['text'], y)
# predicted_svm = text_clf_svm.predict(df_test['text'])
# np.mean(predicted_svm == y_test)



# # features=list(X_train.columns.values)
# # dt = c.fit(X_train, Y_train)
# # Y_pred = c.predict(X_test)
# # score = accuracy_score(Y_test, Y_pred) * 100


# import graphviz
# %matplotlib inline
# dot_data = tree.export_graphviz(dt, out_file=None) 
# graph = graphviz.Source(dot_data) 
# dot_data = tree.export_graphviz(dt, out_file=None, 
#                          feature_names=features,  
#                          class_names= [1,2,3,4],  
#                          filled=True, rounded=True,  
#                          special_characters=True)  
# graph = graphviz.Source(dot_data)
# graph

# with open("fiamge.txt", "w") as f:
#     f = tree.export_graphviz(c, out_file=f)





In [ ]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# from pandas.api.types import is_string_dtype
# from pandas.api.types import is_numeric_dtype
# import scipy.stats as sps
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn import neighbors
# from scipy.stats import *

# c = neighbors.KNeighborsClassifier(n_neighbors=13)
# text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
#                          ('c',neighbors.KNeighborsClassifier(n_neighbors=13)),])

# _ = text_clf_svm.fit(df_train['text'], y)
# predicted_svm = text_clf_svm.predict(df_test['text'])
# np.mean(predicted_svm == y_test)